In [1]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import ipywidgets as widgets
from IPython.display import display, clear_output

# Cost model functions
def linear_model(Q, m, b):
    return m * Q + b

def quadratic_model(Q, a, b, c):
    return a * Q**2 + b * Q + c

def nonlinear_price_model(Q, a, b):
    return a * Q**b

# Function to find all break-even points (across full range)
def find_break_even_points(Q_range_full, TR_full, TC_full):
    diff = TR_full - TC_full
    sign_changes = np.where(np.diff(np.sign(diff)))[0]
    return Q_range_full[sign_changes]

# Function to process and plot
def fit_and_plot_cost_model(cost_data_str, revenue_data_str, quantity_range):
    try:
        # Parse cost input
        cost_lines = cost_data_str.strip().split("\n")
        Q_data, TC_data = zip(*[map(float, line.split(",")) for line in cost_lines])
        Q_data = np.array(Q_data)
        TC_data = np.array(TC_data)

        # Parse revenue input
        rev_lines = revenue_data_str.strip().split("\n")
        Q_rev, P_rev = zip(*[map(float, line.split(",")) for line in rev_lines])
        Q_rev = np.array(Q_rev)
        P_rev = np.array(P_rev)

        # Fit cost models
        popt_lin, _ = curve_fit(linear_model, Q_data, TC_data)
        popt_quad, _ = curve_fit(quadratic_model, Q_data, TC_data)

        # Fit revenue model: price per unit as nonlinear function of Q
        popt_price, _ = curve_fit(nonlinear_price_model, Q_rev, P_rev)

        # Full simulation range for break-even detection (not limited by slider)
        Q_full = np.linspace(1, 2000, 2000)  # start from 1 to capture full range but avoid Q=0
        TR_full = nonlinear_price_model(Q_full, *popt_price) * Q_full
        TC_full = quadratic_model(Q_full, *popt_quad)
        break_even_points = find_break_even_points(Q_full, TR_full, TC_full)

        # Create simulation range for plotting (limited)
        Q_sim = np.linspace(0, quantity_range[1], 400)
        Q_sim_safe = Q_sim.copy()
        Q_sim_safe[Q_sim_safe == 0] = 1e-6  # Avoid zero to prevent divide-by-zero in price model
        P_sim = nonlinear_price_model(Q_sim_safe, *popt_price)
        TR = P_sim * Q_sim_safe
        TC_sim_quad = quadratic_model(Q_sim, *popt_quad)
        profit_quad = TR - TC_sim_quad

        # Max Profit (safe indexing)
        if len(profit_quad) > 0:
            max_profit_quad = np.max(profit_quad)
            Q_max_quad = Q_sim[np.argmax(profit_quad)]
        else:
            max_profit_quad = 0
            Q_max_quad = 0

        # Plotting
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

        # 1. Cost Function
        ax1.scatter(Q_data, TC_data, color='black', label="User Data")
        ax1.plot(Q_sim, linear_model(Q_sim, *popt_lin), 'r-', label=f"Linear Fit: TC = {popt_lin[0]:.2f}Q + {popt_lin[1]:.2f}")
        ax1.plot(Q_sim, quadratic_model(Q_sim, *popt_quad), 'b--', label=f"Quadratic Fit: TC = {popt_quad[0]:.4f}Q² + {popt_quad[1]:.2f}Q + {popt_quad[2]:.2f}")
        ax1.set_title("Cost Function: Linear vs Nonlinear Fit")
        ax1.set_xlabel("Quantity Produced")
        ax1.set_ylabel("Total Cost ($)")
        ax1.legend()
        ax1.grid(True)

        # 2. Profit Function
        ax2.plot(Q_sim, profit_quad, 'purple', linestyle='--', label="Profit (Quadratic Cost)")
        ax2.plot(Q_max_quad, max_profit_quad, 'o', color='purple', label=f"Max Profit: ${max_profit_quad:.2f} at Q={Q_max_quad:.0f}")
        for i, q in enumerate(break_even_points):
            ax2.axvline(q, color='gray', linestyle='dotted', label=f"Break-Even: {q:.0f} units")
        ax2.set_title("Profit Function (Quadratic Cost Only)")
        ax2.set_xlabel("Quantity")
        ax2.set_ylabel("Profit ($)")
        ax2.legend()
        ax2.grid(True)

        # 3. Revenue vs Cost with Profit Highlight
        ax3.plot(Q_sim, TR, 'blue', label="Total Revenue (TR)")
        ax3.plot(Q_sim, TC_sim_quad, 'red', label="Total Cost (Quadratic)")
        ax3.fill_between(Q_sim, TC_sim_quad, TR, where=TR>=TC_sim_quad, color='green', alpha=0.2, label='Profit Region')
        ax3.fill_between(Q_sim, TC_sim_quad, TR, where=TR<TC_sim_quad, color='red', alpha=0.1, label='Loss Region')
        for i, q in enumerate(break_even_points):
            ax3.axvline(q, color='gray', linestyle='dotted', label=f"Break-Even: {q:.0f} units")
        ax3.plot(Q_max_quad, nonlinear_price_model(Q_max_quad, *popt_price) * Q_max_quad, 'o', color='purple', label=f"Max Profit Point")
        ax3.set_title("Revenue vs Cost with Profit Highlight")
        ax3.set_xlabel("Quantity")
        ax3.set_ylabel("Dollars")
        ax3.legend()
        ax3.grid(True)

        plt.tight_layout()
        plt.show()

        # Print results
        print(f"Linear Cost Function: TC(Q) = {popt_lin[0]:.3f}Q + {popt_lin[1]:.2f}")
        print(f"Quadratic Cost Function: TC(Q) = {popt_quad[0]:.4f}Q² + {popt_quad[1]:.2f}Q + {popt_quad[2]:.2f}\n")
        print(f"Quadratic Max Profit: ${max_profit_quad:.2f} at Q = {Q_max_quad:.1f}")
        for i, q in enumerate(break_even_points):
            price = nonlinear_price_model(q, *popt_price)
            print(f"Break-even Quantity {i+1} (Quadratic): {q:.2f} units at Price: ${price:.2f}")

    except Exception as e:
        print("Error:", e)

# Widgets
data_input = widgets.Textarea(
    value="100, 3000\n200, 6000\n300, 11000",
    placeholder='e.g. 100, 3000\n200, 6000\n300, 15000',
    description='Cost Data:',
    layout=widgets.Layout(width='100%', height='100px')
)
revenue_input = widgets.Textarea(
    value="100, 75\n200, 72\n300, 65",
    placeholder='e.g. 100, 75\n200, 72\n300, 65',
    description='Revenue Data:',
    layout=widgets.Layout(width='100%', height='100px')
)
quantity_range_slider = widgets.IntRangeSlider(
    value=[100, 630],
    min=50,
    max=2000,
    step=10,
    description='Quantity Range:',
    continuous_update=False
)

run_button = widgets.Button(description="Fit & Plot Cost Models", button_style='primary')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        fit_and_plot_cost_model(data_input.value, revenue_input.value, quantity_range_slider.value)

run_button.on_click(on_button_clicked)

display(widgets.VBox([data_input, revenue_input, quantity_range_slider, run_button, output]))
